object.__get__(self, instance, owner)


self : Ссылка на экземпляр


instance : Экземпляр класса владельца дескриптора, либо None, если обращаются в контексте класса, а не экземпляра.


owner : Класс владельца дескриптора.

In [ ]:
# создаем дескриптор
class Prop:
    def __init__(self, getter):
        print('self', self) # в момент создания сюда передается новый объект Prop
        print('getter', getter) # в момент создания сюда передается функция custom_sugar
        self.getter = getter # тут лежит функция
    def __get__(self, obj, obj_type=None):
        print(self, obj, obj_type)
        # obj - либо наш объект юзер, либо None если вызов от класса
        # При вызове от объекта у нас вызывается функция. Однако если мы пытаемся обратиться от класса, вернётся объект типа property.
        if obj is None:
            return self
        return self.getter(obj) # в момент обращения к этому реквизиту мы вызываем функцию

In [ ]:
class Class:
    @property
    def original(self):
        return 'original'

    @Prop
    def custom_sugar(self):
        return 'custom sugar'

self <__main__.Prop object at 0x7f3ac2db3128>
getter <function Class.custom_sugar at 0x7f3ac2e23d08>


In [ ]:
obj = Class()
print(obj.original)
print(obj.custom_sugar)
print(Class.custom_sugar)

original
<__main__.Prop object at 0x7f3ac2ea8128> <__main__.Class object at 0x7f3ac2ea88d0> <class '__main__.Class'>
custom sugar
<__main__.Prop object at 0x7f3ac2ea8128> None <class '__main__.Class'>


In [ ]:
class User:
    def __init__(self, first_name, last_name):
        print(1)
        self.first_name = first_name
        print(11)
        self.last_name = last_name
    @property
    def first_name(self):
        print(3)
        return f'{self.__first_name} {self.last_name}'
    @first_name.setter
    def first_name(self, first_name):
        print(2)
        self.__first_name = first_name

amy = User('Amy', 'Jones')
print(amy.first_name)
print(User.first_name)

1
2
11
3
Amy Jones


In [ ]:
class P:
    def __init__(self,x):
        print(1)
        self.x = x
        print(3)

    @property
    def x(self):
        print(4)
        return self.__x

    @x.setter
    def x(self, x):
        print(2)
        if x < 0:
            self.__x = 0
        elif x > 1000:
            self.__x = 1000
        else:
            self.__x = x

Pop = P(100)
# Pop.x

1
2
3


In [22]:
class ClassPropertyDescriptor(object):

    def __init__(self, fget, fset=None):
        print('Инициализируем дискриптор')
        self.fget = fget
        self.fset = fset

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        return self.fget.__get__(obj, klass)()

    def __set__(self, obj, value):
        if not self.fset:
            raise AttributeError("can't set attribute")
        type_ = type(obj)
        # print(self.fset)
        # print(obj) # элемент Bar
        # print(type_)
        # print(self.fset)
        # print(self.fset.__get__(obj, type_))
        print(self.fset.__get__)
        return self.fset.__get__(obj, type_)(value) # создаем bound method тоесть с переданным первым аргументом self

    def setter(self, func):
        print('setter')
        if not isinstance(func, (classmethod, staticmethod)):
            func = classmethod(func)
        self.fset = func
        return self

def classproperty(func):
    if not isinstance(func, (classmethod, staticmethod)):
        func = classmethod(func)

    return ClassPropertyDescriptor(func)


class Bar(object):
    _bar = 1

    @classproperty # в момент вызова bar функция подменилась на объект ClassPropertyDescriptor(func) и автоматически сработал __get__
    def bar(self):
        return self._bar

    @bar.setter # вызываю bar.setter и передаю туда функцию
    def bar(self, value):
        self._bar = value


# test instance instantiation
foo = Bar()
# print(foo.__dir__()) # Атрибут объекта bar это дискриптор
assert foo.bar == 1

baz = Bar()
assert baz.bar == 1

# test static variable
baz.bar = 5
assert foo.bar == 5
print(baz.bar)

# test setting variable on the class
Bar.bar = 50
assert baz.bar == 50
assert foo.bar == 50


Инициализируем дискриптор
setter
<method-wrapper '__get__' of classmethod object at 0x7ff338ca8b70>
5
